In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag_calm import nocalm
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm
import MEC
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

init: ground truth  
hyperparameter: lam = 0.5, beta = 10  
result: ground truthで初期化する場合、すべての構造で正解を出力する。

In [5]:
# init: ground truth
# lam = 0.5, beta = 10

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        #print(data.T@ data / 10000)
        A_est, B_est, info = nocalm(
            R_hat, lam=0.5, beta=10.0, 
            max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
            dtype=torch.float64,A_init = A_true
    )
        
        print("G_est = \n",B_est)
        if MEC.is_in_markov_equiv_class(G_true, B_est): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  2.9562272795925044
SCM 1 : 1.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  3.4562272775313514
SCM 2 : 1.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
Final Loss =  3.956227275470196
SCM 3 : 1.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
Final Loss =  3.9562272754701966
SCM 4 : 1.0
G_est = 
 [[0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]
Final Loss =  4.466010144415651
SCM 5 : 1.0


In [6]:
# init: Cholesky
# lam = 0.5, beta = 10

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        Theta_hat = inv(R_hat)
        A_init = np.linalg.cholesky(Theta_hat)
        #print(data.T@ data / 10000)
        A_est, B_est, info = nocalm(
            R_hat, lam=0.5, beta=10.0, 
            max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
            dtype=torch.float64,A_init = A_init
    )
        
        print("G_est = \n",B_est)
        if MEC.is_in_markov_equiv_class(G_true, B_est): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  2.9584972458104937
SCM 1 : 1.0
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
Final Loss =  3.5032566077246674
SCM 2 : 1.0
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]]
Final Loss =  4.455698710312211
SCM 3 : 0.0
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
Final Loss =  3.9557282784907346
SCM 4 : 1.0
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]]
Final Loss =  4.455685561531782
SCM 5 : 1.0


In [11]:
# X -> Y <- Z
# init: Cholesky
# lam = 0.5, beta = 10

# lams = [0.1, 0.2, 0.5, 0.75, 1]

# for lam in lams:
i = 5
X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = 1)
data = np.array([X, Y, Z]).T
print(data.shape)
R_hat = np.cov(data.T)
Omega_true = np.eye(3)
Theta_hat = inv(R_hat)
A_init = np.linalg.cholesky(Theta_hat)
B_true = np.array([[0,1,2],[0,0,3],[0,0,0]])
print("A_init = \n", A_init)
beta = 5
lam = 0.5

A_true = (np.eye(3) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

#print(data.T@ data / 10000)
A_est, B_est, info = nocalm(
    R_hat, lam=lam, beta=beta, 
    max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
    dtype=torch.float64, A_init = A_init)

print("A_true = \n",A_true)
print("A_est = \n", A_est)
print("G_est = \n", B_est)

print("final loss:", info["final_loss"])
print("final llikelihood:", info["final_llikelihood"])
print("final penalty:", info["final_penalty"])

(10000, 3)
A_init = 
 [[ 2.46448675  0.          0.        ]
 [ 2.04891708  2.42780519  0.        ]
 [-0.81907167 -0.55593119  0.16781583]]
A_true = 
 [[ 1. -1. -2.]
 [ 0.  1. -3.]
 [ 0.  0.  1.]]
likelihood_true =  2.9921669186153252
penalty_true =  1.4999546000700503
f_true =  4.492121518685376
A_true = 
 [[ 1. -1. -2.]
 [ 0.  1. -3.]
 [ 0.  0.  1.]]
A_est = 
 [[ 2.46448675e+00 -4.87223256e-17 -6.60108928e-17]
 [ 2.04891708e+00  2.42780519e+00 -1.63455544e-16]
 [-8.19032448e-01 -5.55390135e-01  1.67815828e-01]]
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]]
final loss: 4.487709740755053
final llikelihood: 2.9918443724612125
final penalty: 1.49586536829384


In [19]:
# X -> Y <- Z
# init: Cholesky
# lam = 0.5, beta = 10

betas = [1, 2, 3, 4, 5]

for beta in betas:
    i = 3
    X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = 1)
    data = np.array([X, Y, Z]).T
    R_hat = np.cov(data.T)
    Omega_true = np.eye(3)
    Theta_hat = inv(R_hat)
    A_init = np.linalg.cholesky(Theta_hat)
    #print("A_init = \n", A_init)
    #beta = 5
    lam = 0.5

    A_true = (np.eye(3) - G_true) @ inv(sqrtm(Omega_true))
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
    print("likelihood_true = ", likelihood_true)
    true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])
    f_true = likelihood_true + true_penalty
    print("penalty_true = ", true_penalty)
    print("f_true = ", f_true)

    #print(data.T@ data / 10000)
    A_est, B_est, info = nocalm(
        R_hat, lam=lam, beta=beta, 
        max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
        dtype=torch.float64, A_init = A_init)

    print("A_true = \n",A_true)
    print("A_est = \n", A_est)
    print("G_est = \n", B_est)

    print("final loss:", info["final_loss"])
    print("final llikelihood:", info["final_llikelihood"])
    print("final penalty:", info["final_penalty"])

likelihood_true =  2.9921669186153315
penalty_true =  0.7615941559557649
f_true =  3.7537610745710963
A_true = 
 [[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]
A_est = 
 [[ 1.19352175e+00 -3.57303442e-03  5.87207865e-11]
 [-3.56344951e-01  6.53984901e-01 -3.55221688e-02]
 [ 9.85923391e-02 -4.04462532e-01  1.02024362e+00]]
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [1. 1. 0.]]
final loss: 3.709357669007444
final llikelihood: 3.2778048284067953
final penalty: 0.4315528406006485
likelihood_true =  2.9921669186153315
penalty_true =  0.9640275800758169
f_true =  3.9561944986911484
A_true = 
 [[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]
A_est = 
 [[ 1.35216641e+00 -1.87827614e-03  1.41434283e-02]
 [-1.67680325e-01  6.41992048e-01 -3.66608688e-02]
 [ 2.09152769e-10 -1.23641283e-01  1.00023636e+00]]
G_est = 
 [[0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
final loss: 4.116039921945007
final llikelihood: 3.780580763545569
final penalty: 0.3354591583994382
likelihood_true =  2.9921669186153315
penalty_true =  0.9

In [ ]:
# ER1
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

# Setup for logging
# Required for printing histories if checkpointing is activated
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s - %(name)s - %(message)s'
)

seed = 1

# Load dataset
n, d = 10000, 4
graph_type, degree = 'ER', 0.5    # ER2 graph
B_scale = 1.0
noise_type = 'gaussian_ev'

Dataset = SyntheticDataset(n, d, graph_type, degree,
                          noise_type, B_scale, seed=seed)
data = Dataset.X
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
A_init = np.linalg.cholesky(Theta_hat)
A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
print("A_init = \n", A_init)
beta = 5
lam = 0.5

# A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

#print(data.T@ data / 10000)
A_est, B_est, info = nocalm(
    R_hat, lam=lam, beta=beta, 
    max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
    dtype=torch.float64, A_init = A_init)

print("A_true = \n",A_true)
print("A_est = \n", A_est)
print("G_est = \n", B_est)

print("final loss:", info["final_loss"])
print("final llikelihood:", info["final_llikelihood"])
print("final penalty:", info["final_penalty"])


(4, 4)
A_init = 
 [[ 2.95647025e+00 -2.25027513e-02  1.73525187e-03 -1.30638421e-02]
 [-4.51844620e-01  8.86768971e-01  1.60052839e-02  4.56455154e-03]
 [ 5.88624373e-01  2.98798640e-01  7.52308433e-01 -1.46795201e-03]
 [ 5.72184528e-01  2.95611953e-01 -5.80529127e-01  5.13069908e-01]]
A_true = 
 [[ 1.         -1.29974793  1.78941959  1.70284777]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
likelihood_true =  3.99747608723802
penalty_true =  1.499997676804117
f_true =  5.497473764042137
A_true = 
 [[ 1.         -1.29974793  1.78941959  1.70284777]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
A_est = 
 [[ 2.70736165e+00  4.57329267e-04 -1.13798780e-04  3.01947138e-04]
 [-1.50558968e-11  8.23618864e-01 -2.75396952e-04 -2.63357244e-09]
 [ 7.38108669e-01  4.94589861e-01  7.02

In [1]:
# ER1, ground truth
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

# Setup for logging
# Required for printing histories if checkpointing is activated
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s - %(name)s - %(message)s'
)

seed = 1

# Load dataset
n, d = 10000, 4
graph_type, degree = 'ER', 0.5    # ER2 graph
B_scale = 1.0
noise_type = 'gaussian_ev'

Dataset = SyntheticDataset(n, d, graph_type, degree,
                          noise_type, B_scale, seed=seed)
data = Dataset.X
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
A_init = np.linalg.cholesky(Theta_hat)
A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
print("A_init = \n", A_init)
beta = 5
lam = 0.5

# A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

#print(data.T@ data / 10000)
A_est, B_est, info = nocalm(
    R_hat, lam=lam, beta=beta, 
    max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
    dtype=torch.float64, A_init = Dataset.B)

print("A_true = \n",A_true)
print("A_est = \n", A_est)
print("G_est = \n", B_est)

print("final loss:", info["final_loss"])
print("final llikelihood:", info["final_llikelihood"])
print("final penalty:", info["final_penalty"])


2025-09-17 14:40:52,154 INFO - numexpr.utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-09-17 14:40:52,156 INFO - numexpr.utils - NumExpr defaulting to 8 threads.


: 